Installing groq and imports


In [ ]:

!pip install groq

from groq import Groq
from termcolor import colored
import requests
import json

API KEY TO CONNECT TO GROQ 


In [ ]:
api_key = "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"

Define the function to connect to groq and config the llm 

In [ ]:
# instanciate groq 
client = Groq(
    api_key=api_key
)
# a function thta will connect to a llm model with groq 
def generate_response(messages, model="llama-3.1-8b-instant", max_tokens=150, temperature=0.7):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )

    response = chat_completion.choices[0].message.content
    return response


This will be the code to execute user input to get champion names, 
based on the champion name, it will fetch to an api with that champion information via json file
the user can then ask about the information about the champion from that file.


In [ ]:
This version can only take in one of the champion and let the user ask about one type of champion that they searched. 

In [ ]:

champion_name = ""

def fetch_champion_data(champion_name):
  url = f"https://ddragon.leagueoflegends.com/cdn/13.1.1/data/en_US/champion/{champion_name}.json"
  try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return data
  except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    return None

def main():
  global champion_name
  champion_name = input("Enter the name of a League of Legends champion: ")

  champion_data = fetch_champion_data(champion_name)
  if champion_data:
    print(champion_data)

    while True:
      user_question = input(f"Ask me something about {champion_name}: ")
      if user_question.lower() == "exit":
        break
      prompt = f"JSON data: {json.dumps(champion_data)}\nQuestion: {user_question}"
      try:
          response = generate_response([{"role": "user", "content": prompt}])
          print(colored(response, 'green'))
      except Exception as e:
          print(f"Error querying Groq: {e}")

if __name__ == "__main__":
  main()

This version will let user get 2 champions and the llm will compare the informations about 2 of the champions that the user will input. 

In [ ]:

champion_data_cache = {}

champion_name = ""

def fetch_champion_data(champion_name):
  if champion_name in champion_data_cache:
    print("Using cached data.")
    return champion_data_cache[champion_name]

  url = f"https://ddragon.leagueoflegends.com/cdn/13.1.1/data/en_US/champion/{champion_name}.json"
  try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    champion_data_cache[champion_name] = data # Cache the data
    return data
  except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    return None

def main():
  global champion_name
  champion1_name = input("Enter the name of the first League of Legends champion: ")
  champion2_name = input("Enter the name of the second League of Legends champion: ")

  champion1_data = fetch_champion_data(champion1_name)
  champion2_data = fetch_champion_data(champion2_name)

  if champion1_data and champion2_data:
    while True:
      user_question = input(f"Ask me something comparing {champion1_name} and {champion2_name}: ")
      if user_question.lower() == "exit":
        break

      prompt = f"Compare and contrast these two champions based on the provided JSON data:\nChampion 1 ({champion1_name}): {json.dumps(champion1_data)}\nChampion 2 ({champion2_name}): {json.dumps(champion2_data)}\nQuestion: {user_question}"
      try:
        response = generate_response([{"role": "user", "content": prompt}])
        print(colored(response, 'green'))
      except Exception as e:
          print(f"Error querying Groq: {e}")

if __name__ == "__main__":
  main()